# COD private API through CallofDuty.py

As of September 2021

---

[Credits to github.com/EthanC](https://github.com/EthanC/CallofDuty.py) <br>For this well made Python client <br><br>
> *"CallofDuty.py is an asynchronous, object-oriented Python wrapper for the Call of Duty API."*

#### Context & objectives

In this notebook, we will test & learn how to use this **--non official, Call of Duty (COD) client to access players' stats**, with a strong inclination towards **Warzone**. <br>
If you didn't know, *Warzone* is a free Battle Royale game, part of the Call of Duty universe, developped by Activision and played by dozen millions of people.<br>
As a player (of Warzone or the more classic online multiplayers modes of the different COD games) you already have access to some stats (score, kills/deaths ratio, rank...) on [my.callofduty.com](https://my.callofduty.com/) but they're poorly put together.<br> 
Cause or consequence, this led to to the creation of a rich ecosytem of --often very popular, websites (cod tracker, wzranked...) promising progression trackers and in depth analysis to players. <br>
As Activision is using a **"private" API** (with no support) on the callofduty.com website and the documentation is very sparse, **we will intend to explore and test the different functions & outputs offered by this wrapper to work with the API.**

#### Resources

AFAIK, the most complete wrappers for COD api are this one and another written in NodeJS [(Github)](https://docs.codapi.dev/getting-started). <br>
A good starting point if you want to get your hands dirty in COD stats would be to read both code. Besides, I would also recommend that you read the [documentation](https://docs.codapi.dev/getting-started) of the NodeJS wrapper as well as this [Postman resources](https://documenter.getpostman.com/view/5519582/SzzgAefq).<br>They will give you a good overall idea of which endpoints, authentification and data are at your disposal.

#### Install and run

Personal preferences here but I'm using miniconda (conda) as a environment manager (could be pyenv etc.) and Poetry for dependency managing and packaging. <br>
In my conda environment I have Python 3.9 (needed for the client), Jupyter and Poetry installed.
1. Create a new project with `poetry new your_project_name` or, if you have already a pre-populated directory, `cd your_existing_project` and then `poetry init`. Cf. [Poetry documentation](https://python-poetry.org/docs/)
2. Install the Call of Duty client : `poetry add callofduty.py`. Poetry will make sure to install all the requirements.
3. Run this notebook : `poetry run jupyter lab`, or `poetry shell` to start a new shell and then `jupyter lab` in the newly opened terminal.
This will ensure you have access to all dependencies, in a custom and clean environment, thus callofduty.py and the notebook perfectly

In [1]:
import asyncio
import os
import dotenv
from pprint import pprint
import callofduty
from callofduty import Mode, Platform, Title, TimeFrame, GameType

#### Login to the API

Two ways to authenticate to COD API. Once you're logged in, you will have access to either private (your info) or protected routes that may supply data for any given user. [Postman](https://docs.codapi.dev/getting-started) to further know what's happening under the hood. <br> 1. Login & password with `callofduty.Login(activision_email, pwd)`. I think it doesn't work anymore since Activision added a reCaptcha (but seems to be doable with the [NodeJS package](https://docs.codapi.dev/getting-started) that's using puppeteers + a plugin to bypass it).<br>2. Single Sign On (sso) `callofduty.Login(SSO_TOKEN)` added recently, that uses a SSO token you get while logging to Activision through your platform of choice (Bnet, Xbox, PS).

In [2]:
# Using SSO
# We're storing our SSO token in an .env file stored locally to separate our config from code (w. python-dotenv). An.env-template file (with help to retrieve token) is provided for you to edit and populate the variable(s)
# callofduty.py client .Login() goes through all the authentification steps and initiate a session to access protected routes
# The client is asynchronous thus the 'await style'
from dotenv import load_dotenv
load_dotenv()
client = await callofduty.Login(sso=os.environ["SSO"])

#### COD universe API, endpoints & client

COD ecosystem is indeed diverse :
- You can have access to one or multiple titles (*Modern Warfare*, *Black Ops Cold War* ...)
- playable locally or more likely online multiplayer
- within every game, several 'modes', e.g the the 'Battle Royale' *Warzone*, also with different maps (called 'modes' also :p)
- through multiple platforms, depending on the game (Steam, Battle Net, Xbox Live...)

Players need to have enabled their visibility to 'on' (obvs. off by default) in their settings so their profile is searchable.<br>
The way the Activision API works is that you generally need to specify, for any given player's, its gamertag associated to a given platform and then the title/mode/sub you want to get data from (a player can have two different gamertags whether he is using BattleNet or Playstation Live).<br>
For in-depth access to player's stats, one generally needs to specify the Platform (e.g. Activision), Title (e.g. Modern Warfare) and the Mode (e.g. multiplayer)<br>
Since it's our focus here, once we identified a player by his gamertag & associated platform, we will usually specify `title = modernwarfare` and `mode = warzone` as Warzone is a free mode developped within the Modern Warfare engine and thus organized this way in the API.<br>
Luckily for us the python wrapper handles the naming in an enums.py file (`Mode, Platform...`) to build the endpoints smoothly, as well as objects/classes (`client.py, match.py, player.py,...`) to work with.

Example of a GET request built in the client to access the API : <br>
Cf. [Postman](https://docs.codapi.dev/getting-started) for details about API's versions & path variables as well as differences between between private, protected and public routes

> `Request("GET",f"api/papi-client/leaderboards/v2/title/{title}/platform/{platform}/time/{timeFrame}/type/{gameType}/mode/{gameMode}/page/{page}",)`

#### Client architecture & (some) useful methods

The table below is quite complete but not exhaustive, took long enough to do ^_^ <br>
Mainly a good way to have the big picture on protected/public routes, useful to gather player Stats.<br>
As mentioned on Postman, routes are either private, public or protected; this will be our plan when we explore the API. <br>

In [101]:
# Do not like when my md table is not aligned to the left
from IPython.core.display import HTML
table_css = 'table {align:left;display:block} '
HTML('<style>{}</style>'.format(table_css))

What you can use | ...depends on .py |     ...depends on .py| ... where it does that call to COD API (https://my.callofduty.com/api.papi-client/.)
:-------------|:--------------|:------------------|:------------------------------------
client.GetPlayer|*returns Player*||
client.SearchPlayers|http.SearchPlayer||crm/cod/v2/platform/platform/username/username/search
client.GetPlayerProfile|http.GetPlayerProfile||stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode
client.GetPlayerMatches   -- if user platform = Activision |http.GetPlayerMatchesDetailed||crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit
client.GetPlayerMatches   -- if user platform = Bnet etc.. |http.GetPlayerMatches||crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime?limit=limit
client.GetPlayerMatchesSummary|http.GetPlayerMatchesDetailed||crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit
client.GetMatchDetails|http.GetMatch||ce/v1/title/title/platform/platform/match/matchId/matchMapEvents
client.GetMatch|*returns Match*||
client.GetFullMatch|http.GetFullMatch||crm/cod/v2/title/title/platform/platform/fullMatch/mode/matchId/language
*The preferred matches endpoint does not support Activision (uno) platform*|*should use (cf. postman):*||crm/cod/v2/title/title/platform/platform/uno/username/matches/mode/start/startTime/end/endTime/details?limit=limit
*re. getMatch endpoint matchMapEvents is for multiplayer only (no Warzone)*|||
*Others: GetMyFriends etc*|||
---|---||
player.profile |client.GetPlayerProfile|http.GetPlayerProfile|stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode
player.matchesSummary|client.GetPlayerMatchesSummary|http.GetPlayerMatchesDetailed|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTime/start/end/endTime/details?limit=limit
player.matches |client.GetPlayerMatches|http.GetPlayerMatches|crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTime/start/end/endTime}?limit=limit
*+ player.loadouts, player.loadoutUnlocks...*|||
---|---||
match.teams|client.GetMatchTeams||ce/v1/title/title/platform/platform/match/matchId/matchMapEvents
match.details|client.GetMatchDetails||ce/v1/title/title/platform/platform/match/matchId/matchMapEvents
*Endpoint matchMapEvents works for multiplayer only (no Warzone)*|||

## Private routes

Not our focus here but once logged in, you have access to private routes related to you own account (only) e.g. your friends' profiles (`client.GetMyFriends()`) and activity, account search visibility, used platform (e.g. Battlenet), identifiers linked to your Activision account etc. <br>
Cf. the test.py where ethanC have listed all the methods available in his client.

In [4]:
# For instance the .GetMyFriends() method, build the private endpoint to retrieve your friends statuses, using the authenticated client (personal credendials & associated gamertag).
friends = await client.GetMyFriends()
for friend in friends:
    print(f"{friend.username}, Online: {friend.online}")

chrissou#9246578, Online: False
Marmiton#4932812, Online: False
Moinolol#4713832, Online: False
nicoyzovitch#7591470, Online: False
ninjawariorbob#7568880, Online: False
Confetti_Seeker#1916728, Online: False


In [5]:
# Here, the client nicely returns a list of dict w. friends' info (client's code indicates the COD API usually returns json or txt)
pprint(friends[0].__dict__)

{'_client': <callofduty.client.Client object at 0x7fe0e438e760>,
 'accountId': '1722124035977126995',
 'avatarUrl': None,
 'identities': [],
 'online': False,
 'platform': <Platform.Activision: 'uno'>,
 'username': 'chrissou#9246578'}


## Public routes

Routes you can access without authentification. Mostly the leaderboards for COD classic ultiplayer modes (MW, BO4), as well as maps & modes available for multiplayer.<br>

### Leaderboards

Global ranking of players by their score, kills, kills deaths (kd) ratios etc.<br>
Searched a lot and in every manner possible but Warzone leaderboard (you can see in in-game :-p) endpoint is protected/out of my reach.<br>
Still, an ex. on how to get the leaderboard from the "Cyber" mode in COD Modern Warfare. The client return a leaderboard object, with entries (players) you can also dive into :

In [6]:
leaderboard = await client.GetLeaderboard(title=Title.ModernWarfare, platform=Platform.BattleNet, gameType=GameType.Core, gameMode="cyber", timeFrame=TimeFrame.AllTime, page=1)
pprint(leaderboard.__dict__, depth=1)
print('\n entries:')
for entry in leaderboard.entries[:3]:
    print(f"{entry.rank}: {entry.username} ({entry.platform.name})")

{'_client': <callofduty.client.Client object at 0x7fe0e438e760>,
 'columns': [...],
 'entries': [...],
 'gameMode': 'cyber',
 'gameType': <GameType.Core: 'core'>,
 'page': 1,
 'pages': 407539,
 'platform': <Platform.BattleNet: 'battle'>,
 'timeFrame': <TimeFrame.AllTime: 'alltime'>,
 'title': <Title.ModernWarfare: 'mw'>}

 entries:
1: 小赵同学#1148917 (BattleNet)
2: BrattySis#6834874 (BattleNet)
3: RNYNN#6664890 (BattleNet)


In the client other methods that access public routes are available such as `.GetPlayerLeaderboard()` (returns the leadeboard'page for a particular user; no Warzone) and `.GetFullMatch()` (Wz compatible)

<a id='go_match'></a>

#### Match details : match/players stats given a certain MatchId

Get detailed stats about a match given a match ID, (moderwarfare/multiplayer or modernware/Warzone etc.)

In [22]:
# w. matchID taken from the Postman example. In this case a battle royale (Warzone) game with 145 players organizezd in teams of 4 (squads).
match = await client.GetFullMatch(Platform.Activision, Title.ModernWarfare, Mode.Warzone, matchId=11763015911965617014)

Returns a dict with a list of dict, every dict being a player-and-his-stats (here 145).<br>
In this match : 145 players, organized in teams of 4 ('br squad').<br>
Our selected player had 0 kills (playerStats.kills), 2 deathes (playerStats.deaths) and was moving 87% of the time (.percentTimeMoving) and the whole team ranked 31 (.teamPlacement)

In [100]:
# One given player returned stats among the 145 (note: should be 37 teams x 4 players = 148 players initially ?).
pprint(match['allPlayers'][2], depth=3)

{'draw': False,
 'duration': 1634000,
 'gameType': 'wz',
 'map': 'mp_don3',
 'matchID': '11763015911965617014',
 'mode': 'br_brquads',
 'player': {'awards': {},
            'brMissionStats': {'missionStatsByType': {},
                               'missionsComplete': 0,
                               'totalMissionWeaponXpEarned': 0.0,
                               'totalMissionXpEarned': 0.0},
            'loadout': [{...}],
            'rank': 54.0,
            'team': 'team_twenty_four',
            'uno': '17641839849440527637',
            'username': 'stuckinatrap'},
 'playerCount': 145,
 'playerStats': {'assists': 0.0,
                 'bonusXp': 0.0,
                 'challengeXp': 0.0,
                 'damageDone': 237.0,
                 'damageTaken': 344.0,
                 'deaths': 2.0,
                 'distanceTraveled': 277414.28,
                 'executions': 0.0,
                 'gulagDeaths': 1.0,
                 'gulagKills': 0.0,
                 'headshots':

## Protected routes

Authentification is mandatory to access those. Good thing is that you can retrieve data for other players (w. visibility setting turned ON)

### Player search

One can play Warzone through PlayStation, PC (BattleNet) or Xbox (also, cross play), hence the username being tied to a platform when searching.<br>
Activision allows to change its own in-game username once in a while (3 months I believe). <br>
Players can share the same name, they differentiate with ending numbers (6 digits for Activision, 4 for Bnet). Max number of players returbed by the COD API is 20.<br>
The client return a list of `player` objects

In [9]:
# For instance, my in-game --changed, username is gentil_renard, I can retrieve it (gentil_renard#3391079) with platform = Activision (translates into 'Uno' when the client builds the route)
results = await client.SearchPlayers(Platform.Activision, "gentil_renard")
for player in results:
    print(f"{player.username} ({player.platform.name})")

# but though I'm playing via Bnet, can't retrieve if I set platform = Bnet
results = await client.SearchPlayers(Platform.BattleNet, "gentil_renard")
for player in results:
    print(f"{player.username} ({player.platform.name})")

# Only works if I use my Bnet gamertag
results = await client.SearchPlayers(Platform.BattleNet, "AMADEVS#1689")
for player in results:
    print(f"{player.username} ({player.platform.name})")   

gentil_renard#3391079 (Activision)
Amadevs#1689 (BattleNet)


In [10]:
# A friend of mine uses a PlayStation
results = await client.SearchPlayers(Platform.PlayStation, "Nicoyzovitch")
for player in results:
    print(f"{player.username} ({player.platform.name})")

# Can also retrieve his name via Activision (nicoyzovitch#7591470) has he never changed his name.
results = await client.SearchPlayers(Platform.Activision, "Nicoyzovitch")
for player in results:
    print(f"{player.username} ({player.platform.name})")

nicoyzovitch (PlayStation)
nicoyzovitch#7591470 (Activision)


In [11]:
# Striking example with 'Huskerrs' (a popular pro player) wannabes . 
# Good thing Activision has an authenticity stamp you can retrieve with player name and phrase (cf. .authenticityStamp in the client)
res = []
for platform in [Platform.Activision, Platform.BattleNet]:
    res.extend(await client.SearchPlayers(Platform.Activision, "HusKerrs"))

for player in res:
    print(f"{player.username} ({player.platform.name})")

Huskerrs (Activision)
HusKerrs#1009786 (Activision)
HusKerrs#1088477 (Activision)
HusKerrs#3209982 (Activision)
HusKerrs#4249229 (Activision)
HusKerrs#4780912 (Activision)
HusKerrs#5139476 (Activision)
HusKerrs#7232956 (Activision)
HusKerrs#7631054 (Activision)
HusKerrs#8490490 (Activision)
HusKerrs#8638305 (Activision)
HusKerrs#8653257 (Activision)
HusKerrs#9624907 (Activision)
HusKerrs#9783265 (Activision)
Huskerrs#2032932 (Activision)
Huskerrs#2058640 (Activision)
Huskerrs#3542853 (Activision)
Huskerrs#7010480 (Activision)
Huskerrs#8797872 (Activision)
Huskerrs#9357694 (Activision)
huskerrs#6821860 (Activision)
Huskerrs (Activision)
HusKerrs#1009786 (Activision)
HusKerrs#1088477 (Activision)
HusKerrs#3209982 (Activision)
HusKerrs#4249229 (Activision)
HusKerrs#4780912 (Activision)
HusKerrs#5139476 (Activision)
HusKerrs#7232956 (Activision)
HusKerrs#7631054 (Activision)
HusKerrs#8490490 (Activision)
HusKerrs#8638305 (Activision)
HusKerrs#8653257 (Activision)
HusKerrs#9624907 (Activisi

## Player Warzone profile

### Profile : client methods

Two ways to retrieve player's profile in the client (same endpoint) : client.GetPlayerProfile or player.profile <br>

##### Profile using client.GetPlayerProfile

In [54]:
# Parameters : platform, username, title, mode
# Endpoint : stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode
profile_using_client = await client.GetPlayerProfile(Platform.BattleNet, "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone)
pprint(profile_using_client, depth=2)

{'engagement': None,
 'level': 348.0,
 'levelXpGained': 6940.0,
 'levelXpRemainder': 35960.0,
 'lifetime': {'accoladeData': {...},
              'all': {...},
              'itemData': {...},
              'map': {},
              'mode': {...},
              'scorestreakData': {...}},
 'maxLevel': 1.0,
 'maxPrestige': 0.0,
 'p': 0.0,
 'paragonId': 0.0,
 'paragonRank': 0.0,
 'platform': 'battle',
 'prestige': 23.0,
 'prestigeId': 0.0,
 's': 0.0,
 'title': 'mw',
 'totalXp': 1325315.0,
 'type': 'wz',
 'username': 'AMADEVS#1689',
 'weekly': {'all': {...}, 'map': {}, 'mode': {...}}}


##### Profile using player.profile

In [85]:
# Getting player object first, as defined in player.py
# Parameters : platform, username

player = await client.GetPlayer(Platform.BattleNet, "AMADEVS#1689")
print(f"{player.username} ({player.platform.name})")

# then, calling the .profile method
# Parameters : title, mode
# Endpoint : stats/cod/v1/title/title/platform/platform/gamer/username/profile/type/mode
profile_using_player = await player.profile(Title.ModernWarfare, Mode.Warzone)
pprint(profile_using_player, depth=2)

AMADEVS#1689 (BattleNet)
{'engagement': None,
 'level': 348.0,
 'levelXpGained': 6940.0,
 'levelXpRemainder': 35960.0,
 'lifetime': {'accoladeData': {...},
              'all': {...},
              'itemData': {...},
              'map': {},
              'mode': {...},
              'scorestreakData': {...}},
 'maxLevel': 1.0,
 'maxPrestige': 0.0,
 'p': 0.0,
 'paragonId': 0.0,
 'paragonRank': 0.0,
 'platform': 'battle',
 'prestige': 23.0,
 'prestigeId': 0.0,
 's': 0.0,
 'title': 'mw',
 'totalXp': 1325315.0,
 'type': 'wz',
 'username': 'AMADEVS#1689',
 'weekly': {'all': {...}, 'map': {}, 'mode': {...}}}


### Profile : stats returned by COD API

If mode was set to Multiplayer, data would be similar but stats related to Modern Warfare game / multiplayers modes & maps.<br>
The client returns a dict : the key "data" from sent COD API json. No means to change timespan Lifetime or Weekly.<br>
Examples of stats available and where to find them, instead of doing a giant print ^_^

`dict['username']`, `dict['platform']`, `dict['level']`) : given username & associated platform, level of the Player (1 to 1000) <br>
`dict['level'])` & `dict['levelXpGained']` : are shared between Modern Warfare multiplayer & Warzone. Can't remember if `['prestige']` depends on Level or challenges that has to be done in game.<br>
`dict['lifetime']` : expandable, **lifetime** consolidated stats of the player.<br>
`dict['weekly']` : expandable, **weekly** consolidated stats of the player.<br>
`dict['lifetime']` and `dict['weekly']` can be further explored. They have similar keys (**all**, **mode**, **map**), except Lifetime having 3 more (**itemData**, **scorestreakData**, **accoladeData**). <br>

##### Lifetime stats

`dict['lifetime']['all']` : consolidated, lifetime stats, no matter the map/mode you're playing on.<br>
>`..['properties']['accuracy']` : Warzone lifetime accuracy no matter the map (classic Battle Royale, Plunder etc.).<br>
`..['properties']['wins']` : total number of wins no matter the map.<br>
`..['properties']['gamesPlayed']` : total number of games played.<br>
`..['properties']['kdRatio']` : overall kills/deaths ratio.<br>
and many others : bestKillStreak, headshots, bestKD...

`dict['lifetime']['mode']` : lifestime stats, given a specified mode (br, br_dmz, br_all, arena, koth..).<br>
>`..['br']['properties']` : player stats related specifically to mode ?.<br>
`..['br_dmz']['properties']` : player stats related specifically to mode ? .<br>
`..['br_all']['properties']` : player stats related specifically to I guess consolidated br solo duo, trio, quads ?<br>
other mods are available : koth, arena, though belonging to MW Multiplayer and not Warzone, whereas mode is set to Warzone in our call.
>>  `..['br']['properties']['kills']` : number of kills for mode.<br>
`..['br']['properties']['topTen']` : number of top ten for mode.<br>
`..['br']['properties']['wins']` : number of wins for mode.<br>
and a few others : revives, kills, deathes, kdRatio, gamesPlayed...

`dict['lifetime']['map']` : Empty for me, not sure exactly why, but tested with a player that plays MW/Multiplaters, and still empty. <br>

`dict['lifetime']['itemData']` : additional key for lifetime dictionary (aka doesn't exist for 'weekly'). Lifetime stats related to weapon & equipment<br>
> Inside, several stats equipement/weapon related, per category of equipement/weapon and item wise

`dict['lifetime']['scorestreakData']` : additional key for 'lifetime' dictionary. Lifetime stats related to 'special' equipment. <br>
> Among others, how many uav, airstrike, sentry guns... used, for some of them might return the corresponding n of kills but not sure.

`dict['lifetime']['accoladeData']` : additional key for 'lifetime' dictionary. Even more diverse and surprising lifetime stats, all modes together ^_^. <br>
> Lifestats such as timeWatchingKillcams, timeWatchingKillcams, comebackKills... though not sure they're whether fully accurate or updated. E.g deathsFromBehind is 5 for me whereas I'm sure died a lot more that way :D

##### Weekly Stats

`dict['lifetime']` and `dict['weekly']` have similar keys (**all**, **mode**, **map**) to 'Lifetime' except the latter having 3 more (**itemData**, **scorestreakData**, **accoladeData**).<br>
Meaning that you won't have stats about weapons used, 'kill streaks' or other listed above.<br>
When calling the key 'mode' for level 'weekly', the API details the stats for each specific mode of Warzone (battle royale duos, trios, quads (br_all?), plunder) ; a granularity not available at 'lifetime' level.<br>

In [105]:
pprint(profile_using_player['weekly']['mode'].keys())

dict_keys(['br_dmz_plndtrios', 'br_dmz_plunquad', 'br_brtrios', 'br_all', 'br_brduos'])


## Matches

Retrieve Player's last Matchs IDs.<br>
If you remember well, you can then explore them with [Match Details](#go_match)

##### Matches using client.GetPlayerMatches

In [14]:
# Parameters : platform, username, title, mode, start, end, limit
# Endpoint if platform = Activision              : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit through method http.GetPlayerMatchesDetailed
# Endpoint if platform = Bnet, PlayStation, Xbox : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTime/start/end/endTime?limit=limit         through method http.GetPlayerMatches
matches_using_client = await client.GetPlayerMatches(Platform.Activision, "gentil_renard#3391079", Title.ModernWarfare, Mode.Warzone, limit=2)
for match in matches_using_client:
    pprint(match.__dict__)

{'_client': <callofduty.client.Client object at 0x7fe0e438e760>,
 'id': 15571015748597522761,
 'platform': <Platform.Activision: 'uno'>,
 'title': <Title.ModernWarfare: 'mw'>}
{'_client': <callofduty.client.Client object at 0x7fe0e438e760>,
 'id': 18288595769756463118,
 'platform': <Platform.Activision: 'uno'>,
 'title': <Title.ModernWarfare: 'mw'>}


##### Matches using player.matches

In [15]:
# Getting player object first, as defined in player.py
# Parameters : platform, username
player = await client.GetPlayer(Platform.BattleNet, "AMADEVS#1689")
print(f"{player.username} ({player.platform.name})")

# then, calling the .matches method
# Parameters : title, mode, start, end, limit
# Endpoint if platform = Activision              : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit through method http.GetPlayerMatchesDetailed
# Endpoint if platform = Bnet, PlayStation, Xbox : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/startTime/start/end/endTime?limit=limit         through method http.GetPlayerMatches
# Returns a [n match objects]
matches_using_player = await player.matches(Title.ModernWarfare, Mode.Warzone, limit=2)
for match in matches_using_player:
    pprint(match.__dict__)

AMADEVS#1689 (BattleNet)
{'_client': <callofduty.client.Client object at 0x7fe0e438e760>,
 'id': 15571015748597522761,
 'platform': <Platform.BattleNet: 'battle'>,
 'title': <Title.ModernWarfare: 'mw'>}
{'_client': <callofduty.client.Client object at 0x7fe0e438e760>,
 'id': 18288595769756463118,
 'platform': <Platform.BattleNet: 'battle'>,
 'title': <Title.ModernWarfare: 'mw'>}


## Matches Summary

##### Summary using client.GetPlayesMatchesSummary

In [16]:
# Parameters : title, mode, limit
# Endpoint : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit
# Returns stats for n last matches, all + per type  br, plunder ...

summary = await client.GetPlayerMatchesSummary(Platform.BattleNet, "AMADEVS#1689", Title.ModernWarfare, Mode.Warzone, limit=20)# does not work if I use player = await client.GetPlayer(Platform.Activision, "gentil_renard") cf. Postman 'Warzone by Uno ID' ?
pprint(summary, depth=2)

{'all': {'assists': 17.0,
         'avgLifeTime': 413.78504672897196,
         'damageDone': 23127.0,
         'damageTaken': 12926.0,
         'deaths': 87.0,
         'distanceTraveled': 7370153.370000001,
         'executions': 0.0,
         'gulagDeaths': 12.0,
         'gulagKills': 6.0,
         'headshotPercentage': 0.2033898305084746,
         'headshots': 12.0,
         'kdRatio': 0.6781609195402298,
         'kills': 59.0,
         'killsPerGame': 2.95,
         'matchesPlayed': 20.0,
         'nearmisses': 0.0,
         'objectiveBrCacheOpen': 60.0,
         'objectiveBrDownEnemyCircle1': 21.0,
         'objectiveBrDownEnemyCircle2': 7.0,
         'objectiveBrDownEnemyCircle3': 1.0,
         'objectiveBrKioskBuy': 17.0,
         'objectiveBrMissionPickupTablet': 13.0,
         'objectiveDestroyedEquipment': 7.0,
         'objectiveDestroyedVehicleMedium': 1.0,
         'objectiveLastStandKill': 27.0,
         'objectivePlunderCashBloodMoney': 1.0,
         'objectiveReviver'

##### Summary using player.matchesSummary

In [17]:
# Getting player object first, as defined in player.py
# Parameters : platform, username
player = await client.GetPlayer(Platform.BattleNet, "AMADEVS#1689")
print(f"{player.username} ({player.platform.name})")

# then, calling the .matchesSummary method
# Parameters : title, mode, limit
# Endpoint : crm/cod/v2/title/title/platform/platform/gamer/username/matches/mode/start/startTime/end/endTime/details?limit=limit
# Returns stats for n last matches, all + per type  br, plunder ...
summary = await player.matchesSummary(Title.ModernWarfare, Mode.Warzone, limit=20) # does not work if I use player = await client.GetPlayer(Platform.Activision, "gentil_renard") cf. Postman Warzone by Uno ID ?
pprint(summary, depth=1)

AMADEVS#1689 (BattleNet)
{'all': {...},
 'br_brduos': {...},
 'br_brtrios': {...},
 'br_dmz_plndtrios': {...}}


## Addon : let's add extra methods to the client so we handle a "new" API route

As listed on Postman, COD API has an another endpoint (see code below) currently not used by the client. I was curious whether or not it would give us anything different. <br>
Turned out the additional method return n matches stats (summary + details per match), similar to what you would get with client.GetPlayersMatchesDetailed().<br>
The only difference is that it takes UnoID as a parameter e.g. 12309926, different from Activision User ID e.g 'gentil_renard#3391079'. Additionally something you cannot do with other routes.

In [20]:
# we're adding additional methods in the Call of Duty .py client, without modifying the package.
# Import the Class we want to modify, without touching the client's package

from callofduty.client import Client
from callofduty.http import HTTP
from callofduty.http import Request
import urllib.parse

# define additonal method in callofduty.http HTTP Class
# Endpoint extracted from Postman, currently not supported by the client (but turned out not to be mandatory tbh)

async def GetWithUnoID(self, platform, username, title, mode, limit, startTimestamp, endTimeStamp):
    return await self.Send(
        Request(
            "GET",
            f"api/papi-client/crm/cod/v2/title/{title}/platform/{platform}/uno/{urllib.parse.quote(username)}/matches/{mode}/start/{startTimestamp}/end/{endTimeStamp}/details?limit={limit}",
        )
    )
# define additional method in callofduty.client Client Class
# We chose not to differentiate between 'summary'and 'matches" entries when returning the results --as done in the the client (aka return them both).

async def GetMatchesWithUnoID(self, platform: Platform, username, title: Title, mode: Mode, **kwargs):
    limit = kwargs.get("limit", 10)
    startTimestamp = kwargs.get("startTimestamp", 0)
    endTimestamp = kwargs.get("endTimestamp", 0)

    data = (
        await self.http.GetWithUnoID(
            platform.value,
            username,
            title.value,
            mode.value,
            limit,
            startTimestamp,
            endTimestamp
        )
    )["data"]

    return data

# let's add our additional methods into respective Classes, at runtime

Client.GetMatchesWithUnoID = GetMatchesWithUnoID
HTTP.GetWithUnoID = GetWithUnoID

In [19]:
# Now we can use our new method as if it was in the package ^_^
matches_with_unoID = await client.GetMatchesWithUnoID(Platform.Activision, "12309926", Title.ModernWarfare, Mode.Warzone, limit=2)
pprint(matches_with_unoID, depth=3)

 sent request : 
api/papi-client/crm/cod/v2/title/mw/platform/uno/uno/12309926/matches/wz/start/0/end/0/details?limit=2
{'matches': [{'draw': False,
              'duration': 1657000,
              'gameType': 'wz',
              'map': 'mp_don4',
              'matchID': '2604020260401454789',
              'mode': 'br_brtrios',
              'player': {...},
              'playerCount': 151,
              'playerStats': {...},
              'playlistName': None,
              'privateMatch': False,
              'rankedTeams': None,
              'teamCount': 51,
              'utcEndSeconds': 1633229017,
              'utcStartSeconds': 1633227360,
              'version': 1},
             {'draw': False,
              'duration': 1626000,
              'gameType': 'wz',
              'map': 'mp_don4',
              'matchID': '13089921240256125878',
              'mode': 'br_brduos',
              'player': {...},
              'playerCount': 150,
              'playerStats': {...}

## Addon : Let's test the client using asyncronous features